In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import itertools
from more_itertools import set_partitions

In [3]:
pd.options.display.max_rows = 12
truck_speed = 60
working_hours = 12
m = 5 # Number of trucks
#get a matrix of distances between locations
df = pd.read_csv('dist_vologda_matrix.csv', sep=',')
df = df.iloc[:, 1:]
df = df / 1000
df

,АЗС №35400 Кичменгский Городок,АЗС №35401 Великий Устюг-1,АЗС №35402 Великий Устюг-2,АЗС №35403 Дунай,АЗС №35404 Нюксеница,АЗС №35405 Борисово-Судское,АЗС №35406 Журавлево,АЗС №35407 Грязовец-1,АЗС №35408 Грязовец-2,АЗС №35409 Савино,...,АЗС №35454 Октябрьский пр.-1 (Вологда),АЗС №35455 Октябрьский пр.-2 (Вологда),АЗС №35456 Маяковская,АЗС №35457 Краснодонцев,АЗС №35458 Северсталь,АЗС №35459 Олимпийская,АЗС №35460 Северное шоссе-1,АЗС №35461 Северное шоссе-2,АЗС № 35462 Аксеново,Нефтебаза
0,0.0000,99.2272,101.8513,139.2135,154.2522,777.2635,102.4328,520.5020,517.8863,266.7014,...,610.3886,610.3028,607.3283,609.0579,611.0852,611.0605,604.2684,605.7555,502.9826,477.9325
1,99.0684,0.0000,4.9465,134.5201,149.5589,748.0318,5.5280,491.2703,488.6546,241.7276,...,581.1570,581.0711,578.0967,579.8263,581.8535,581.8288,575.0368,576.5238,473.7510,448.7008
2,101.8148,5.0688,0.0000,137.2665,152.3053,750.7781,3.0302,494.0166,491.4010,244.4739,...,583.9033,583.8174,580.8430,582.5726,584.5999,584.5752,577.7831,579.2702,476.4973,451.4472
3,139.2135,134.6791,137.3031,0.0000,15.0961,613.5690,137.8846,356.8075,354.1918,107.2648,...,446.6942,446.6083,443.6339,445.3635,447.3908,447.3660,440.5740,442.0610,339.2882,314.2380
4,154.2522,149.7178,152.3419,15.0961,0.0000,613.8947,152.9234,357.1332,354.5176,107.5905,...,447.0199,446.9341,443.9596,445.6892,447.7165,447.6918,440.8997,442.3868,339.6139,314.5638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,611.7861,582.6174,585.2414,447.9957,448.3214,191.2351,585.8229,181.8523,179.2367,345.2112,...,10.1276,10.0417,7.2967,3.3719,10.8241,0.0000,6.1623,4.6720,107.2118,142.1894
60,605.7962,576.6275,579.2515,442.0057,442.3315,185.0682,579.8330,175.8624,173.2467,339.2213,...,6.8726,6.7867,3.8122,4.8070,7.5691,6.8095,0.0000,1.5045,101.2218,136.1995
61,607.1467,577.9779,580.6020,443.3562,443.6820,186.4186,581.1835,177.2129,174.5972,340.5717,...,8.2230,8.1372,5.1627,3.5396,8.9196,5.5422,1.5229,0.0000,102.5723,137.5500
62,504.6641,475.4954,478.1194,340.8736,341.1994,274.3706,478.7009,74.7303,72.1146,238.0892,...,107.4957,107.4099,104.4354,106.1650,108.1923,108.1676,101.3755,102.8626,0.0000,35.0674


In [4]:
#remove locations, which cannot be served in 12 hours (driver can't get there and come back in 12 hours)

to_remove = ((2 * (df.values[-1,:] / truck_speed) < working_hours) == False).nonzero()[0]
df = df.drop(df.index[to_remove])
df = df.drop(df.columns[to_remove], axis=1)
df = df.reset_index(drop=True)
df

,АЗС №35403 Дунай,АЗС №35404 Нюксеница,АЗС №35405 Борисово-Судское,АЗС №35407 Грязовец-1,АЗС №35408 Грязовец-2,АЗС №35409 Савино,АЗС №35410 Сазоново-1,АЗС №35411 Сазоново-2,АЗС №35412 Вытегра,АЗС №35413 Липник,...,АЗС №35454 Октябрьский пр.-1 (Вологда),АЗС №35455 Октябрьский пр.-2 (Вологда),АЗС №35456 Маяковская,АЗС №35457 Краснодонцев,АЗС №35458 Северсталь,АЗС №35459 Олимпийская,АЗС №35460 Северное шоссе-1,АЗС №35461 Северное шоссе-2,АЗС № 35462 Аксеново,Нефтебаза
0,0.0000,15.0961,613.5690,356.8075,354.1918,107.2648,619.0077,618.5201,629.3182,472.6696,...,446.6942,446.6083,443.6339,445.3635,447.3908,447.3660,440.5740,442.0610,339.2882,314.2380
1,15.0961,0.0000,613.8947,357.1332,354.5176,107.5905,619.3335,618.8458,629.6440,472.9954,...,447.0199,446.9341,443.9596,445.6892,447.7165,447.6918,440.8997,442.3868,339.6139,314.5638
2,615.1980,615.5238,0.0000,349.0547,346.4390,512.4135,142.9597,142.4720,165.8965,193.2966,...,187.4480,187.3622,184.7850,188.3816,183.6881,191.1772,185.1426,186.6296,274.4141,309.3918
3,356.1259,356.4516,349.7479,0.0000,3.7018,253.3414,355.1867,354.6990,373.5276,216.8790,...,182.8731,182.7873,179.8128,181.5424,183.5697,183.5450,176.7529,178.2400,75.4671,45.6714
4,353.5507,353.8764,347.1727,3.7422,0.0000,250.7662,352.6115,352.1238,370.9524,214.3038,...,180.2979,180.2120,177.2376,178.9672,180.9945,180.9698,174.1777,175.6648,72.8919,43.0962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,447.9957,448.3214,191.2351,181.8523,179.2367,345.2112,196.6738,196.1861,332.9386,176.2900,...,10.1276,10.0417,7.2967,3.3719,10.8241,0.0000,6.1623,4.6720,107.2118,142.1894
52,442.0057,442.3315,185.0682,175.8624,173.2467,339.2213,190.5069,190.0192,326.9487,170.3001,...,6.8726,6.7867,3.8122,4.8070,7.5691,6.8095,0.0000,1.5045,101.2218,136.1995
53,443.3562,443.6820,186.4186,177.2129,174.5972,340.5717,191.8574,191.3697,328.2991,171.6506,...,8.2230,8.1372,5.1627,3.5396,8.9196,5.5422,1.5229,0.0000,102.5723,137.5500
54,340.8736,341.1994,274.3706,74.7303,72.1146,238.0892,279.8093,279.3216,346.5072,189.8586,...,107.4957,107.4099,104.4354,106.1650,108.1923,108.1676,101.3755,102.8626,0.0000,35.0674


In [5]:
idx = [10, 23, 46,  7, 11, 40, 12, 53, 16,  2, 45, 38,  1, 34, 43, 44,  3,
       14, 15, 24, 41, 22, 19, 37, 33, 54, 39,  6, 36,  0, 49,  4, 48, 30,
       50, 27, 17, 18,  9, 21]

In [6]:
from numpy.random import default_rng

#shrink array
# df = df.iloc[40:56,40:56]
rng = default_rng()
#idx = [10, 23, 46, 7, 11, 40, 12, 53, 16, 2, 45, 38, 1, 34, 43, 44, 3, 14, 15, 24, 41, 22, 19, 37, 33, 54, 39, 6, 
       #36, 0, 49, 4, 48, 30, 50, 27, 17, 18, 9, 21]
print(idx)
df = df.drop(df.index[idx])
df = df.drop(df.columns[idx], axis=1)
df = df.reset_index(drop=True)
N = len(df.values[0])
df

[10, 23, 46, 7, 11, 40, 12, 53, 16, 2, 45, 38, 1, 34, 43, 44, 3, 14, 15, 24, 41, 22, 19, 37, 33, 54, 39, 6, 36, 0, 49, 4, 48, 30, 50, 27, 17, 18, 9, 21]


,АЗС №35409 Савино,АЗС №35412 Вытегра,АЗС №35417 Барское,АЗС №35427 Марьинское,АЗС №35433 Сокол,АЗС №35434 Верховажье,АЗС №35436 Вожега,АЗС №35437 Харовск,АЗС №35439 Козленская,АЗС №35440 Мудрова,АЗС №35443 Маршала Конева-1,АЗС №35450 Кирилловское шоссе-1,АЗС №35455 Октябрьский пр.-2 (Вологда),АЗС №35459 Олимпийская,АЗС №35460 Северное шоссе-1,Нефтебаза
0,0.0000,526.3698,183.4339,205.5501,186.5665,122.8875,310.9966,240.3297,210.0147,206.7060,211.7736,338.3356,343.6599,344.4176,337.6256,211.2896
1,528.0211,0.0000,344.4233,321.5271,356.9127,543.2065,509.9042,432.2983,328.4836,328.7171,330.2426,326.7767,332.1010,332.8587,326.0666,329.7586
2,183.5979,342.9359,0.0000,22.1162,12.4894,198.7833,151.0916,87.8750,26.5808,23.2721,28.3397,154.9017,160.2260,160.9837,154.1917,27.8557
3,206.6218,320.8910,23.0239,0.0000,35.5133,221.8072,174.1155,110.8989,14.5443,12.1694,16.3032,132.8568,138.1811,138.9388,132.1468,15.8192
4,186.8042,355.4094,12.4735,34.5897,0.0000,201.9896,138.6263,75.4097,39.0543,35.7457,40.8132,167.3752,172.6995,173.4573,166.6652,40.3292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,339.1860,326.9134,155.5882,132.6920,168.0776,354.3714,306.6797,243.4632,134.8893,135.9759,135.0299,0.0000,6.7514,7.4895,0.6975,136.1642
12,345.3000,333.0274,161.7021,138.8060,174.1915,360.4854,312.7937,249.5771,141.0033,142.0899,141.1438,6.1660,0.0000,9.8063,6.8114,142.2782
13,345.2112,332.9386,161.6133,138.7172,174.1027,360.3966,312.7049,249.4884,140.9145,142.0011,141.0550,7.4471,10.0417,0.0000,6.1623,142.1894
14,339.2213,326.9487,155.6234,132.7272,168.1128,354.4067,306.7150,243.4984,134.9246,136.0112,135.0651,1.4624,6.7867,6.8095,0.0000,136.1995


In [7]:
dest = np.arange(N - 1)
dest

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [8]:
import math
perm_n = math.factorial(N-1)
print(perm_n)

1307674368000


In [9]:
def GetLength(route):
#     if len(route) < 2:
#         return df.values[-1, route[0]] + df.values[route[0], -1]
    length = df.values[-1, route[0]]
    for i in range(1, len(route)):
        length += df.values[route[i-1], route[i]]
    length += df.values[route[-1], -1]
    
    return length

In [10]:
it = itertools.permutations(dest)
sum(1 for ignore in it)

KeyboardInterrupt: 

In [ ]:
from itertools import combinations
combs = [np.split(dest, idx) for n_splits in range(m) for idx in combinations(range(1, len(dest)), n_splits)]

import datetime
start = datetime.datetime.now()
print(start)
i = 0
min_dist = 10000
long = False
for perm in itertools.permutations(dest):
    i += 1
    if i == 1000:
        print("{:}| {:}".format(i, datetime.datetime.now()))
    #print("\r")
    combs = [np.split(perm, idx) for n_splits in range(m) for idx in combinations(range(1, len(perm)), n_splits)]
    dist = 0
    for comb in combs:
        for sub in comb:
            if GetLength(sub) / truck_speed <= working_hours:
                dist += GetLength(sub)
            else: 
                long = True
                break
        if long:
            long = False
            continue
        if dist < min_dist:
            min_dist = dist
            result = comb

finish = datetime.datetime.now()
print(finish)
duration = finish - start
print("Total Hours: ", duration.total_seconds() / 3600)
print("Shortest solution: ", result)
print("It's total distance is ", min_dist)

2022-06-08 16:28:55.366603
1000| 2022-06-08 16:29:53.237673


In [ ]:
# to_del = []
# for i in range(len(combs)):
#     for j in range(len(combs[i])):
#         length = GetLength(combs[i][j])
#         if (length / truck_speed) > working_hours:
#             to_del.append(i)
# len(to_del)

In [ ]:
# new_combs = []
# for i in range(len(combs)):
#     if i not in to_del:
#         new_combs.append(combs[i])
# combs = new_combs
# len(combs)

In [ ]:
# import datetime
# start = datetime.datetime.now()
# print(start)

# short_dist = 10000
# shortest = 0
# for i in range(len(combs)):
#     print(i)
#     dist = 0
#     route = []
#     for j in range(len(combs[i])):
#         short_perm = combs[i][j]
#         for perm in itertools.permutations(combs[i][j]):
#             if GetLength(perm) < GetLength(short_perm):
#                 short_perm = perm
#         dist += GetLength(short_perm)
#         route.append(short_perm)
#     if dist < short_dist:
#         short_dist = dist
#         shortest = i
#         short_route = route
# finish = datetime.datetime.now()
# print(finish)
# duration = finish - start
# print("Total Hours: ", duration.total_seconds() / 3600)
# print("Shortest solution: ", short_route)
# print("It's total distance is ", short_dist)

In [ ]:
# dist = 0
# for i in range(len(short_route)):
#     dist += GetLength(short_route[i])
#     print("Truck #{:} | Time: {:} | Distance: {:}".format(i, GetLength(short_route[i]) / truck_speed, GetLength(short_route[i])))
# dist   

In [ ]:
# GetLength((8, 3, 2, 4, 7, 6, 5, 9))